In [1]:
import pandas as pd
import json

In [2]:
stories = pd.read_csv('stories_description.csv', header=0, sep=',')
stories.head()

,story_id,story_json
0,127,"{""guid"":""770a5bae-0e3f-4a6b-b924-bd87bd51a038""..."
1,254,"{""guid"":""64f4c9ef-647b-4e04-b4d4-02297e939388""..."
2,865,"{""guid"":""3482206b-d223-4aec-92ba-0150055cd68a""..."
3,1491,"{""guid"":""5f4a9215-01de-4777-b70f-a18899db8f1c""..."
4,598,"{""guid"":""ed8754bd-67be-4fa1-9289-5508d96f1fa4""..."


In [3]:
stories['info'] = stories.story_json.apply(lambda x: json.loads(x.replace("\\\\\"", r"\\\\\"")))
stories = stories.drop('story_json', axis=1)

In [4]:
def story_proc(d, ind, types):
    types.add(d['type'])
    if d['type'] == 'label':
        if 'text' not in d['properties']:
           print(d['properties'].keys())
    if 'content' not in d:
        return 
    for c in d['content']:
        story_proc(c, ind + 2, types)

In [278]:
s = set()
for i in range(stories.shape[0]):
data[213]    story_proc(stories.iloc[i, 1], 0, s)
print(s)

dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'corner_radius', 'background', 'size', 'padding', 'margin', 'border'])
dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'corner_radius', 'background', 'size', 'padding', 'margin', 'border'])
dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'template_identifier', 'corner_radius', 'background', 'size', 'padding', 'margin', 'border'])
dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'background', 'size'])
dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'background', 'size'])
dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'background', 'size'])
dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'background'])
dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'background', 'size', 'margin'])
dict_keys(['font_size', 'font_type', 'font_color', 'alignment', 'template_identifier', 'corner_radius', 'background', 'size', 'padding'

{'selection-layout', 'root', 'story-page', 'rating', 'layout-linear', 'label', 'image'}

In [5]:
def story_gather(d, images, texts):
    if d['type'] == 'root':
        if 'name' in d['properties']:
            texts.append(d['properties']['name'])
        if 'icon_url' in d['properties']:
            images.append(d['properties']['icon_url'])
        if 'background' in d['properties']:
            if 'image_url' in d['properties']['background']:
                images.append(d['properties']['background']['image_url'])
        if 'icon_background' in d['properties']:
            if 'image_url' in d['properties']['icon_background']:
                images.append(d['properties']['icon_background']['image_url'])
    elif d['type'] == 'story-page':
        if 'background' in d['properties']:
            if 'image_url' in d['properties']['background']:
                images.append(d['properties']['background']['image_url'])
    elif d['type'] == 'layout-linear':
        pass
        # nothing ? (only gradients as backgrounds, bunch of technical properties, nothing of information)
    elif d['type'] == 'selection-layout':
        pass
        # nothing ? (the same as layout-linear)
    elif d['type'] == 'rating':
        pass
        # nothing ? (the same, and only 2 objects have it)
    elif d['type'] == 'image':
        images.append(d['properties']['url'])
    elif d['type'] == 'label':
        if 'text' in d['properties']:
            texts.append(d['properties']['text'])
    if 'content' in d:
        for c in d['content']:
            story_gather(c, images, texts)    

In [6]:
def StoryGet(story):
    images, texts = [], []
    story_gather(story, images, texts)
    images = list(set(images))
    texts = list(set(texts))
    return images, texts

In [7]:
stories['data'] = stories['info'].apply(StoryGet)
stories = stories.drop('info', axis=1)
stories['texts'] = stories['data'].apply(lambda x: x[1])
stories['images'] = stories['data'].apply(lambda x: x[0])
stories.drop('data', axis=1, inplace=True)

In [16]:
stories['text'] = stories.texts.apply(lambda x: " ".join(x))

In [17]:
stories.text

0      Как изменить ПИН-код Изменить пин-код Введите ...
1      В приложении Тинькофф Как пополнить карту «Стр...
2      Тинькофф Инвестиции 2.0 У нас хорошие новости ...
3      Персонализация — глобальный фэшн-тренд, от кот...
4      Где поиграть Где попробовать Где попрыгать Нео...
                             ...                        
954    Авиабилет за мили Нажмите «Использовать мили»....
955    Выберите покупку, которую хотите компенсироват...
956    Сравните открывшиеся реквизиты с данными в кви...
957    Если хотите вводить ПИН-код, смените приоритет...
958    Как найти банкомат На карте переключитесь на «...
Name: text, Length: 959, dtype: object

In [18]:
stories.drop('texts', axis=1, inplace=True)

In [19]:
stories.to_csv('stories_ImagesTexts.csv', sep=',', index=False)

In [331]:
stories.iloc[324, 2]

['https://static2.tinkoff.ru/portfolio/stories/images/online_kursi_27-03_4.jpg',
 'https://static2.tinkoff.ru/portfolio/stories/images/online_kursi_27-03_5.jpg',
 'https://static2.tinkoff.ru/portfolio/stories/images/online_kursi_27-03_zahod.jpg',
 'https://static2.tinkoff.ru/portfolio/stories/images/online_kursi_27-03_3.jpg',
 'https://static2.tinkoff.ru/portfolio/stories/images/online_kursi_27-03_2.jpg',
 'https://static2.tinkoff.ru/portfolio/stories/images/online_kursi_27-03_1.jpg']